In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [2]:
### Load hg19 gene,TSS information

In [3]:
gene_annotation<- read.table("../../ANALYSIS/gene_annot_jun10.txt",header=T)

In [4]:
head(gene_annotation)

,ENSGID,NAME,CHR,START,END,TSS
,<chr>,<chr>,<int>,<int>,<int>,<int>
1,ENSG00000186092,OR4F5,1,69091,70008,69091
2,ENSG00000235249,OR4F29,1,367640,368634,367640
3,ENSG00000185097,OR4F16,1,621059,622053,622053
4,ENSG00000187634,SAMD11,1,860260,879955,860260
5,ENSG00000188976,NOC2L,1,879584,894689,894689
6,ENSG00000187961,KLHL17,1,895967,901095,895967


In [5]:
## Trans

In [6]:
trans_lead <- read.table("../../ANALYSIS/METSIM_LEAD.TAB",header=T)

In [7]:
#gene to variant mapping

In [8]:
gene_variant <- data.frame()

for (i in 1:nrow(trans_lead) ){
   
line_q <- trans_lead[i,]
marker_q <- as.character(line_q[1])
chr_q <- as.integer(line_q[2])
start_q<- as.integer(line_q[6])
end_q <- as.integer(line_q[7])
lead_q <- as.integer(line_q[5])
position_q <- as.integer(line_q[3])
tmp_df <- gene_annotation%>%filter(CHR==chr_q & START >= start_q & END <= end_q)%>%mutate(INDEX=marker_q,WINDOW=lead_q,POSITION=position_q,TSS_DISTANCE=abs(TSS-POSITION))
gene_variant <- rbind(gene_variant,tmp_df)
}

In [9]:
###MAGMA

In [10]:
trans_magma <- read.table("../../ANALYSIS/POPS/METSIM_MAGMA_OUTPUT.TAB.genes.out",header=T)%>%rename(ENSGID=GENE)
gene_variant_1 <- merge(gene_variant,trans_magma,by="ENSGID")%>%select(ENSGID,NAME,CHR.x,START.x,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,P)%>%rename(CHR=CHR.x,START=START.x,MAGMA_P=P)

In [11]:
head(gene_variant_1)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,MAGMA_P
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,7.2766e-04
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,1.0045e-03
3,ENSG00000010322,NISCH,3,52489134,52527087,52489134,rs2276824,2,52637486,148352,3.0592e-05
4,ENSG00000010327,STAB1,3,52529354,52558511,52529354,rs2276824,2,52637486,108132,3.8966e-06
5,ENSG00000016391,CHDH,3,53846362,53880417,53880417,rs2276824,2,52637486,1242931,9.4306e-02
6,ENSG00000016864,GLT8D1,3,52728505,52740048,52740048,rs2276824,2,52637486,102562,1.7761e-06


In [12]:
## Colocalization

In [13]:
trans_coloc <- read.table("../../ANALYSIS/GTEX/METSIM_COLOC.txt",header=T)
trans_coloc <- merge(gene_variant_1,trans_coloc,by="NAME")%>%select(ENSGID,Tissue,H4_abf)

In [14]:
gene_variant_2<- data.frame()
for (i in 1:nrow(gene_variant_1) ){
line_q <- gene_variant_1[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(trans_coloc%>%filter(ENSGID==ensg_q))
if (genep>0){
c_df <- trans_coloc%>%filter(ENSGID==ensg_q)%>%select(Tissue,H4_abf)%>%mutate(weight = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
pip_coloc <- max(c_df$H4_abf)
coloc_weight <- max(as.double((c_df%>%filter(H4_abf==pip_coloc))$weight))
}else {
pip_coloc <- 0
coloc_weight <- 0
}
tmp_df <- gene_variant_1[i,]%>%mutate(coloc_pip=pip_coloc,coloc_WT=coloc_weight)
gene_variant_2 <- rbind(gene_variant_2,tmp_df)
}



In [15]:
head(gene_variant_2)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,MAGMA_P,coloc_pip,coloc_WT
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,7.2766e-04,0,0
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,1.0045e-03,0,0
3,ENSG00000010322,NISCH,3,52489134,52527087,52489134,rs2276824,2,52637486,148352,3.0592e-05,0,0
4,ENSG00000010327,STAB1,3,52529354,52558511,52529354,rs2276824,2,52637486,108132,3.8966e-06,0,0
5,ENSG00000016391,CHDH,3,53846362,53880417,53880417,rs2276824,2,52637486,1242931,9.4306e-02,0,0
6,ENSG00000016864,GLT8D1,3,52728505,52740048,52740048,rs2276824,2,52637486,102562,1.7761e-06,0,0


In [16]:
##SMR

In [17]:
trans_smr <- read.table("../../ANALYSIS/SMR//METSIM_SMR.tab",header=T)%>%rename(ENSGID=probeID)

In [18]:
gene_variant_3 <- data.frame()
for (i in 1:nrow(gene_variant_2) ){
line_q <- gene_variant_2[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(trans_smr%>%filter(ENSGID==ensg_q))
if (genep>0){
smr_df <- trans_smr%>%filter(ENSGID==ensg_q)%>%select(filename,p_SMR)%>%mutate(weight_smr = case_when(grepl("Artery|Adipose|Heart", filename) ~ "1",TRUE ~ '0.8'))
p_smr <- min(smr_df$p_SMR)
smr_weight <- max(as.double((smr_df%>%filter(p_SMR==p_smr))$weight_smr))
}else {
p_smr <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_2[i,]%>%mutate(p_SMR=p_smr,SMR_WT=smr_weight)
gene_variant_3 <- rbind(gene_variant_3,tmp_df)
}


In [19]:
head(gene_variant_3)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,MAGMA_P,coloc_pip,coloc_WT,p_SMR,SMR_WT
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,7.2766e-04,0,0,NA,0.0
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,1.0045e-03,0,0,NA,0.0
3,ENSG00000010322,NISCH,3,52489134,52527087,52489134,rs2276824,2,52637486,148352,3.0592e-05,0,0,0.1799546000,1.0
4,ENSG00000010327,STAB1,3,52529354,52558511,52529354,rs2276824,2,52637486,108132,3.8966e-06,0,0,0.1485833000,0.8
5,ENSG00000016391,CHDH,3,53846362,53880417,53880417,rs2276824,2,52637486,1242931,9.4306e-02,0,0,0.0818636600,1.0
6,ENSG00000016864,GLT8D1,3,52728505,52740048,52740048,rs2276824,2,52637486,102562,1.7761e-06,0,0,0.0005232975,0.8


In [20]:
## eQTL overlaps

In [21]:
trans_eqtl <- read.table("../../ANALYSIS/GTEX/METSIM_EQTL_OVERLAPS.txt",fill=TRUE,row.names=NULL,header=TRUE)

In [22]:
gene_variant_4 <- data.frame()
for (i in 1:nrow(gene_variant_3) ){
line_q <- gene_variant_3[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(trans_eqtl%>%filter(Transcript==ensg_q))
if (genep>0){
smr_df <- trans_eqtl%>%filter(Transcript==ensg_q)%>%select(Tissue,P_EQTL)%>%mutate(weight_lookup = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
p_smr <- min(smr_df$P_EQTL)
smr_weight <- max(as.double((smr_df%>%filter(P_EQTL==p_smr))$weight_lookup))
}else {
p_smr <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_3[i,]%>%mutate(P_EQTL=p_smr,EQTL_WT=smr_weight)
gene_variant_4 <- rbind(gene_variant_4,tmp_df)
}


In [23]:
head(gene_variant_4,1)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,MAGMA_P,coloc_pip,coloc_WT,p_SMR,SMR_WT,P_EQTL,EQTL_WT
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,0.00072766,0,0,NA,0,NA,0


In [24]:
### LDSC(Expression modifier Score)

In [25]:
trans_ldsc <- read.table("../../ANALYSIS/GTEX/METSIM_LDSC_RESULTS.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [26]:
head(trans_ldsc,2)

,GENE,Enrichment_p,Coefficient_z_score,Tissue,TSS
,<chr>,<dbl>,<dbl>,<chr>,<int>
1,CCDC90B,0.3255285,0.9855832,Adrenal_Gland,82997450
2,PCF11,0.4580465,0.7435084,Artery_Tibial,82868030


In [27]:
gene_variant_5 <- data.frame()
for (i in 1:nrow(gene_variant_4) ){
line_q <- gene_variant_4[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(trans_ldsc%>%filter(GENE==ensg_q))
if (genep>0){
ldsc_df <- trans_ldsc%>%filter(GENE==ensg_q)%>%select(Tissue,Enrichment_p)%>%mutate(weight_ldsc = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
p_ldsc <- min(ldsc_df$Enrichment_p)
smr_weight <- max(as.double((ldsc_df%>%filter(Enrichment_p==p_ldsc))$weight_ldsc))
}else {
p_ldsc <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_4[i,]%>%mutate(LDSC_P=p_ldsc,LDSC_WT=smr_weight)
gene_variant_5 <- rbind(gene_variant_5,tmp_df)
}


In [28]:
head(gene_variant_5)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,MAGMA_P,coloc_pip,coloc_WT,p_SMR,SMR_WT,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,7.2766e-04,0,0,NA,0.0,NA,0,NA,0.0
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,1.0045e-03,0,0,NA,0.0,NA,0,7.674931e-05,1.0
3,ENSG00000010322,NISCH,3,52489134,52527087,52489134,rs2276824,2,52637486,148352,3.0592e-05,0,0,0.1799546000,1.0,NA,0,1.579683e-05,1.0
4,ENSG00000010327,STAB1,3,52529354,52558511,52529354,rs2276824,2,52637486,108132,3.8966e-06,0,0,0.1485833000,0.8,NA,0,1.539592e-05,1.0
5,ENSG00000016391,CHDH,3,53846362,53880417,53880417,rs2276824,2,52637486,1242931,9.4306e-02,0,0,0.0818636600,1.0,NA,0,9.866012e-01,0.8
6,ENSG00000016864,GLT8D1,3,52728505,52740048,52740048,rs2276824,2,52637486,102562,1.7761e-06,0,0,0.0005232975,0.8,NA,0,2.495341e-03,1.0


In [29]:
#gene_variant_5 <- gene_variant_4

In [30]:
##EMS overlaps

In [31]:
trans_ems <- read.table("../../ANALYSIS/EMS_OVERLAP/METSIM_OVERLAP_EMS.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [32]:
head(trans_ems,2)

,CHROMOSOME,POSITION,GENE,EMS_N,Tissue,scaled_EMS
,<int>,<int>,<chr>,<dbl>,<chr>,<dbl>
1,11,1022327,ENSG000001849565,101.41,Artery_Aorta,0.0002388518
2,11,104916866,ENSG00000204397,149.29,Adipose_Subcutaneous,0.0087101294


In [33]:
gene_variant_6 <- data.frame()
for (i in 1:nrow(gene_variant_5) ){
line_q <- gene_variant_5[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(trans_ems%>%filter(GENE==ensg_q))
if (genep>0){
ems_df <- trans_ems%>%filter(GENE==ensg_q)%>%select(Tissue,scaled_EMS)%>%mutate(weight_ems = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
s_ems <- max(ems_df$scaled_EMS)
ems_weight <- max(as.double((ems_df%>%filter(scaled_EMS==s_ems))$weight_ems))
}else {
s_ems <- 0
ems_weight <-0 
}
tmp_df <- gene_variant_5[i,]%>%mutate(EMS_SCORE=s_ems,EMS_WT=ems_weight)
gene_variant_6 <- rbind(gene_variant_6,tmp_df)
}

In [34]:
head(gene_variant_6,2)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,coloc_pip,coloc_WT,p_SMR,SMR_WT,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,⋯,0,0,NA,0,NA,0,NA,0,0.0000000000,0
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,⋯,0,0,NA,0,NA,0,7.674931e-05,1,0.0003131613,1


In [35]:
### POPS
stand01 <- function(x){(x-min(x,na.rm = TRUE))/(max(x,na.rm = TRUE)-min(x,na.rm = TRUE))}

In [36]:
pops_ems <- read.table("../../ANALYSIS/POPS/METSIM_POPS.txt",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(POPS=stand01(PoPS_score))

In [37]:
head(pops_ems,2)

,ENSG,GENE,PoPS_score,POPS
,<chr>,<chr>,<dbl>,<dbl>
1,ENSG00000137509,PRCP,-0.9374609,0.4197745
2,ENSG00000165490,C11orf82,-0.6625387,0.4414417


In [38]:
gene_variant_7 <- data.frame()
for (i in 1:nrow(gene_variant_6) ){
line_q <- gene_variant_6[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(pops_ems%>%filter(ENSG==ensg_q))
if (genep>0){
pops_df <- pops_ems%>%filter(ENSG==ensg_q)%>%select(POPS)
pops_score <- as.double(pops_df)
}else {
pops_score <- 0 
}
tmp_df <- gene_variant_6[i,]%>%mutate(POPS_SCORE=pops_score)
gene_variant_7 <- rbind(gene_variant_7,tmp_df)
}

In [39]:
head(gene_variant_7,2)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,coloc_WT,p_SMR,SMR_WT,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,⋯,0,NA,0,NA,0,NA,0,0.0000000000,0,0.5255572
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,⋯,0,NA,0,NA,0,7.674931e-05,1,0.0003131613,1,0.5150698


In [40]:
## DOWNSTREAMER

In [41]:
down_df <- read.table("../../ANALYSIS/DOWNSTREAMER//METSIM_Downstreamer.txt",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(DOWNSTREAMER_P=pnorm(q=ZSCORE, lower.tail=TRUE))

In [42]:
head(down_df,2)

,ENSG,NAME,ZSCORE,DOWNSTREAMER_P
,<chr>,<chr>,<dbl>,<dbl>
1,ENSG00000086696,HSD17B2,1.972356,0.9757155
2,ENSG00000140943,MBTPS1,1.841270,0.9672090


In [43]:
gene_variant_8 <- data.frame()
for (i in 1:nrow(gene_variant_7) ){
line_q <- gene_variant_7[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(down_df%>%filter(ENSG==ensg_q))
if (genep>0){
downs_df <- down_df%>%filter(ENSG==ensg_q)%>%select(DOWNSTREAMER_P)
p_down <- min(downs_df$DOWNSTREAMER_P)
}else {
p_down <- NA
}
tmp_df <- gene_variant_7[i,]%>%mutate(DOWN_P=p_down)
gene_variant_8 <- rbind(gene_variant_8,tmp_df)
}

In [44]:
gene_variant_8%>%filter(NAME=="ADIPOQ")

ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,p_SMR,SMR_WT,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE,DOWN_P
<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000181092,ADIPOQ,3,186560463,186576252,186560463,rs199938283,4,186552469,7994,⋯,NA,0,1.03e-09,1,0.01528739,1,0.2546231,0.8,0.9540819,0.5841135


In [45]:
## Epi MAp

In [46]:
epi_map <- read.table("../../ANALYSIS/EPIMAP//EPIMAP.tab",fill=TRUE,row.names=NULL,header=TRUE)

In [47]:
head(epi_map,2)

,gene,score,group
,<chr>,<chr>,<chr>
1,ENSG00000008128,0.0882523589664035,Adipose
2,ENSG00000078808,0.0823467903667026,Adipose


In [48]:
gene_variant_9 <- data.frame()
for (i in 1:nrow(gene_variant_8) ){
line_q <- gene_variant_8[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(epi_map%>%filter(gene==ensg_q))
if (genep>0){
epi_df <- epi_map%>%filter(gene==ensg_q)%>%select(score)
epi_score <- max(epi_df$score)
}else {
epi_score <- 0
}
tmp_df <- gene_variant_8[i,]%>%mutate(EPIMAP_SCORE=epi_score)
gene_variant_9 <- rbind(gene_variant_9,tmp_df)
}

In [49]:
gene_variant_9%>%filter(NAME=="ADIPOQ")

ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,SMR_WT,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE,DOWN_P,EPIMAP_SCORE
<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ENSG00000181092,ADIPOQ,3,186560463,186576252,186560463,rs199938283,4,186552469,7994,⋯,0,1.03e-09,1,0.01528739,1,0.2546231,0.8,0.9540819,0.5841135,0.416221218865092


In [50]:
## Genehancer

In [51]:
gh <- read.table("../../ANALYSIS/GeneHancer/GENEHANCER.TAB",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(CHROM=as.integer(om))%>%rename(START=name,END=start,SCORE=end)%>%
select(-om)%>%mutate(SCORE=stand01(SCORE))

Warning message in mask$eval_all_mutate(quo):
"NAs introduced by coercion"


In [52]:
head(gh,2)

,START,END,SCORE,CHROM
,<int>,<int>,<dbl>,<int>
1,133518045,133518161,0.00617284,6
2,51630569,51633727,0.24691358,13


In [53]:
gene_variant_10 <- data.frame()
for (i in 1:nrow(gene_variant_9) ){
line_q <- gene_variant_9[i,]
chr_q <- as.integer(line_q[3])
start_q<- as.integer(line_q[4])
end_q <- as.integer(line_q[5])
genep<- nrow(gh%>%filter(CHROM==chr_q & START >= start_q & END <= end_q))
if (genep>0){
gh_df <- gh%>%filter(CHROM==chr_q & START >= start_q & END <= end_q)%>%select(SCORE)
gh_score <- max(gh_df$SCORE)
}else {
gh_score <- 0
}
tmp_df <- gene_variant_9[i,]%>%mutate(GH_SCORE=gh_score)
gene_variant_10 <- rbind(gene_variant_10,tmp_df)
}

In [54]:
gene_variant_10%>%filter(NAME=="ADIPOQ")

ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,P_EQTL,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE,DOWN_P,EPIMAP_SCORE,GH_SCORE
<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
ENSG00000181092,ADIPOQ,3,186560463,186576252,186560463,rs199938283,4,186552469,7994,⋯,1.03e-09,1,0.01528739,1,0.2546231,0.8,0.9540819,0.5841135,0.416221218865092,0.5740741


In [55]:
## CS2GO SCORE

In [56]:
cs2g <- read.table("../../ANALYSIS/cS2G/TRANS_CS2G.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [57]:
head(cs2g,2)

,CHROMOSOME,POSITION,GENENAME,SCORE,INFO
,<int>,<int>,<chr>,<dbl>,<chr>
1,4,96437606,UNC5C,1.0,|Roadmap=1|PCHiC_Jung=1|ABC=1
2,18,46361441,SMAD7,0.5,|PCHiC_Javierre=1|PCHiC_Jung=1|Cicero=1


In [58]:
gene_variant_11 <- data.frame()
for (i in 1:nrow(gene_variant_10) ){
line_q <- gene_variant_10[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(cs2g%>%filter(GENENAME==ensg_q))
if (genep>0){
cs2g_df <- cs2g%>%filter(GENENAME==ensg_q)%>%select(SCORE)
score <- max(cs2g_df$SCORE)
}else {
score <- NA
}
tmp_df <- gene_variant_10[i,]%>%mutate(CS2G_SCORE=score)
gene_variant_11 <- rbind(gene_variant_11,tmp_df)
}

In [59]:
head(gene_variant_11)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE,DOWN_P,EPIMAP_SCORE,GH_SCORE,CS2G_SCORE
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,⋯,0,NA,0.0,0.0000000000,0,0.5255572,0.97996707,0.412455677986145,0.6759259,0.80
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,⋯,0,7.674931e-05,1.0,0.0003131613,1,0.5150698,0.96314888,0.936377247174581,0.1481481,0.93
3,ENSG00000010322,NISCH,3,52489134,52527087,52489134,rs2276824,2,52637486,148352,⋯,0,1.579683e-05,1.0,0.0000000000,0,0.6401346,0.56006577,0.865026626322005,0.4444444,1.00
4,ENSG00000010327,STAB1,3,52529354,52558511,52529354,rs2276824,2,52637486,108132,⋯,0,1.539592e-05,1.0,0.0000000000,0,0.6836916,0.93621599,0.897401365565091,0.1419753,1.00
5,ENSG00000016391,CHDH,3,53846362,53880417,53880417,rs2276824,2,52637486,1242931,⋯,0,9.866012e-01,0.8,0.0000000000,0,0.4700521,0.01067889,0.700117923864504,0.0000000,1.00
6,ENSG00000016864,GLT8D1,3,52728505,52740048,52740048,rs2276824,2,52637486,102562,⋯,0,2.495341e-03,1.0,0.0000000000,0,0.6712111,0.62797546,0.661368333631092,0.0000000,1.00


In [60]:
colnames(gene_variant_11)

[1] "ENSGID"       "NAME"         "CHR"          "START"        "END"         
 [6] "TSS"          "INDEX"        "WINDOW"       "POSITION"     "TSS_DISTANCE"
[11] "MAGMA_P"      "coloc_pip"    "coloc_WT"     "p_SMR"        "SMR_WT"      
[16] "P_EQTL"       "EQTL_WT"      "LDSC_P"       "LDSC_WT"      "EMS_SCORE"   
[21] "EMS_WT"       "POPS_SCORE"   "DOWN_P"       "EPIMAP_SCORE" "GH_SCORE"    
[26] "CS2G_SCORE"

In [61]:
head(gene_variant_11,2)

,ENSGID,NAME,CHR,START,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,⋯,EQTL_WT,LDSC_P,LDSC_WT,EMS_SCORE,EMS_WT,POPS_SCORE,DOWN_P,EPIMAP_SCORE,GH_SCORE,CS2G_SCORE
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,ENSG00000010318,PHF7,3,52444673,52457657,52444673,rs2276824,2,52637486,192813,⋯,0,NA,0,0.0000000000,0,0.5255572,0.9799671,0.412455677986145,0.6759259,0.80
2,ENSG00000010319,SEMA3G,3,52467069,52479101,52479101,rs2276824,2,52637486,158385,⋯,0,7.674931e-05,1,0.0003131613,1,0.5150698,0.9631489,0.936377247174581,0.1481481,0.93


In [62]:
length(na.omit(gene_variant_11$POPS_SCORE))

[1] 152

In [63]:
## GENE PRIORITY SCORE

In [64]:
library('metap')

Warning message:
"package 'metap' was built under R version 4.2.1"


In [65]:
gene_priority <- data.frame()
for (i in 1:nrow(gene_variant_11) ){
line_q <- gene_variant_11[i,] 
p_g_raw <- na.omit(c(line_q$MAGMA_P,line_q$p_SMR,line_q$P_EQTL,line_q$LDSC_P,line_q$DOWN_P))
if (length(p_g_raw)<2){
p_g <- p_g_raw
} else {
p_g <- -log10(as.double(allmetap(p_g_raw,method="sumz")$p))}
S_G <- sum(as.double(na.omit(c(line_q$coloc_pip,line_q$EMS_SCORE,line_q$POPS_SCORE,line_q$EPIMAP_SCORE,line_q$GH_SCORE,line_q$CS2G_SCORE))))
C_S <- sum(as.double(na.omit(c(line_q$coloc_WT,line_q$SMR_WT,line_q$EQTL_WT,line_q$LDSC_WT,line_q$EMS_WT)))+
           length(as.double(na.omit(line_q$MAGMA_P,line_q$POPS_SCORE,line_q$DOWN_P,line_q$EPIMAP_SCORE,line_q$GH_SCORE,line_q$CS2G_SCORE))))/11
GP_SCORE <- ((as.double(p_g)+as.double(S_G))*as.double(C_S))/log2(line_q$TSS_DISTANCE)
tmp_df <- gene_variant_11[i,]%>%mutate(GP_SCORE=as.double(GP_SCORE))
gene_priority  <- rbind(gene_priority,tmp_df)
}

In [66]:
gene_priority <- arrange(gene_priority%>%group_by(WINDOW)%>% mutate(mx = max(GP_SCORE)) %>% arrange(desc(mx), desc(GP_SCORE)) %>% select(-mx),CHR,POSITION)%>%
select(NAME,CHR,POSITION,INDEX,GP_SCORE,TSS_DISTANCE,MAGMA_P,p_SMR,P_EQTL,LDSC_P,DOWN_P,coloc_pip,EMS_SCORE,POPS_SCORE,GH_SCORE,CS2G_SCORE,coloc_WT,SMR_WT,EQTL_WT,LDSC_WT,EMS_WT)

Adding missing grouping variables: `WINDOW`


In [67]:
write.table(gene_priority,"../../ANALYSIS/GENE_PRIORITY/METSIM_GENE_PRIORITY.TAB",row.names=FALSE,quote=FALSE,sep="\t")

In [139]:
top3_genes <- gene_priority%>%group_by(WINDOW) %>% slice_max(order_by = GP_SCORE, n = 3)

In [140]:
write.table(top3_genes,"../../ANALYSIS/GENE_PRIORITY/METSIM_GENE_PRIORITY_TOP3.TAB",row.names=FALSE,quote=FALSE,sep="\t")